# Machine Learning Zoomcamp Homeworks

## Week 10

In this homework, we'll deploy the churn prediction model from session 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.

### Bulding the image

Clone the course repo if you haven't:

```
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
```

Go to the `course-zoomcamp/05-deployment/code` folder and 
execute the following:


```bash
docker build -t churn-model:v001 .
```

> **Note:** If you have troubles building the image, you can 
> use the image I built and published to docker hub:
> `agrigorev/zoomcamp-model:churn-v001`

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 churn-model:v001
```

And in another terminal, execute `predict-test.py` file:

```bash
python predict-test.py
```

You should see this:

```
{'churn': False, 'churn_probability': 0.3257561103397851}
not sending promo email to xyz-123
```

Now you can stop the container running in Docker.


### Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

### Question #1: Version of kind

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [1]:
# answer to question #1

!kind --version

kind version 0.11.1


### Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster

### Question #2: Verifying that everything works

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's `CLUSTER-IP` of the service that is already running there?

In [5]:
# answer to question #2

!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


As we can see, `10.96.0.1` is the `CLUSTER-IP` of the service listed by above command.

### Question #3: Uploading the image to kind

To be able to use the docker image we previously created (`churn-model:v001`),
we need to register it with kind.

What's the command we need to run for that?

__Answer:__ We should use this command to register the image with kind:

If you used docker build manually:

```bash
kind load docker-image churn-model:v001
```

Or if you used docker pull to get the pre-built image:

```bash
kind load docker-image agrigorev/zoomcamp-model:churn-v001
```

In [7]:
# answer to question #3

!kind load docker-image agrigorev/zoomcamp-model:churn-v001

Image: "agrigorev/zoomcamp-model:churn-v001" with ID "sha256:9114c930ca5fe93a530341d3bf5f125861000c0f19b79fdafebc66c1bfa32c61" not yet present on node "kind-control-plane", loading...


In order to check if the image is registered, we can use the following command in our terminal:

```bash
docker exec -ti kind-control-plane bash
```

and then run the following command:

```bash
crictl images
```

That should give a list of images that are currently registered, similar to this screenshot:

![loaded docker images](resources/week-10/loaded_images.png "Loaded Docker Images")

### Question #4: Creating a deployment

Now let's create a deployment (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: churn
spec:
  selector:
    matchLabels:
      app: churn
  template:
    metadata:
      labels:
        app: churn
    spec:
      containers:
      - name: churn
        image: <Image>
        resources:
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: <Port>
```

Replace `<Image>` and `<Port>` with the correct values.

What is the value for `<Port>`?

__Answer:__ The value for `<Port>` should be the same as the one we used in `Dockerfile` script in `05-deployment/code` folder to expose container port, which is __`9696`__.

The `<Image>` tag in above lines for `deployment.yaml` file should also be replaced with `churn-model:v001` if you built the docker image manually, or `agrigorev/zoomcamp-model:churn-v001` if you used `docker pull` to get the pre-built image.

### Question #5: Pod name

Apply this deployment:

```yaml
kubectl apply -f deployment
```

Now get a list of running pods.
What's the name of the pod that just started?

In [8]:
# answer to question #5

!kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
churn-b495897dd-rcnj8   1/1     Running   0          79s


The above command gives us __churn-b495897dd-rcnj8__ as the "NAME" for the pod. Note that the pod name is generated by Kubernetes and will be different if you run the command on different machines.

### Question #6: Creating a service 

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.

__Answer:__ We need to write `churn` as the value for `<???>` in `service.yaml` file (the same value as in `deployment.yaml` for app label of our pod). I used `churn` as the name of the service instead of `<Service name>` tag. The `<PORT>` tag in `targetPort` line should also be replaced with __9696__ (the exposed port in `Dockerfile` script).

In [9]:
# get running service

!kubectl get services

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
churn        LoadBalancer   10.96.83.193   <pending>     80:31361/TCP   3m21s
kubernetes   ClusterIP      10.96.0.1      <none>        443/TCP        24h


### Testing the service locally

We can do it by forwarding the 9696 port on our computer to the port 80 on the service:

```bash
kubectl port-forward service/churn 9696:80
```

Run `predict-test.py` from session 5 to verify that everything is working.

After doing the port-forward, running `predict-test.py` from session 5 confirms the same response, now coming from Kubernetes:

![Kubernetes response](resources/week-10/kubernetes_response.png "Kubernetes Response")